In [1]:
import duckdb
import plotly.express as px
import pandas as pd
import plotly.figure_factory as ff

In [2]:
conn = duckdb.connect("../flood_forecasting.duckdb", read_only=True)

In [3]:
conn.sql("SHOW ALL TABLES").show()

┌───────────────────┬──────────────┬──────────────────────────────────────┬─────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────

In [4]:
query = "SELECT * FROM flood_forecasting.main_marts.dim_sites LIMIT 10"
conn.sql(query).show()

┌──────────┬──────────────────────────────────────────────────┬─────────────┬──────────────┬─────────────────────┬──────────┐
│ site_id  │                   station_name                   │  latitude   │  longitude   │ drainage_area_sq_mi │ huc_code │
│ varchar  │                     varchar                      │   double    │    double    │       double        │  int64   │
├──────────┼──────────────────────────────────────────────────┼─────────────┼──────────────┼─────────────────────┼──────────┤
│ 05013600 │ St. Mary River near St. Mary MT                  │  48.7260111 │ -113.4356806 │               130.0 │ 10010002 │
│ 05018000 │ St. Mary Canal at intake near Babb MT            │ 48.85277778 │ -113.4169444 │                NULL │ 10010002 │
│ 05018500 │ St. Mary Canal at St. Mary Crossing near Babb MT │ 48.94696667 │ -113.3753306 │                NULL │ 10010002 │
│ 05019000 │ St. Mary Canal at Hudson Bay Divide nr Browning  │  48.9949864 │ -113.0775923 │                NULL │ 100

In [5]:
query = "SELECT * FROM flood_forecasting.main_marts.dim_catchment_attributes LIMIT 10"
conn.sql(query).show()

┌──────────┬───────────────────────────────────────────┬─────────────┬──────────────┬──────────┬─────────────────────┬───────────────────────┬─────────────┬────────────┬────────────┬───────────┬────────────────────┬────────────────────┬─────────────────────────┬───────────────────────┬──────────────┬──────────────┬──────────────┬────────────────┬────────────────────────┬───────────────────────────────────────────────────────────────────────────────────┬──────────────────┬──────────────────┬──────────────────┬──────────────────────┬───────────────────┬──────────────────┬──────────────────┬───────────────────┬──────────────────────┬────────────────────────┬─────────────────────────┬──────────────────────┬────────────────────┬─────────────────────┬─────────────────────┬───────────────────────┬─────────────────────┬──────────────────────┐
│ site_id  │               station_name                │  latitude   │  longitude   │ huc_code │ drainage_area_sq_km │ is_reference_hcdn2009 │ elev_mean_

In [6]:
query = """
    SELECT a.site_id
    FROM flood_forecasting.main_marts.dim_sites AS a
    RIGHT JOIN flood_forecasting.main_marts.dim_catchment_attributes AS b ON a.site_id = b.site_id
    WHERE a.site_id IS NULL
"""
conn.sql(query).show()

┌─────────┐
│ site_id │
│ varchar │
├─────────┤
│ 0 rows  │
└─────────┘



In [7]:
query = """
    SELECT *
    FROM flood_forecasting.main_marts.dim_catchment_attributes AS a
    WHERE site_id = '05013500'
"""
conn.sql(query).show()

┌─────────┬──────────────┬──────────┬───────────┬──────────┬─────────────────────┬───────────────────────┬─────────────┬────────────┬────────────┬───────────┬──────────────────┬─────────────────┬─────────────────────────┬───────────────────────┬──────────────┬──────────────┬──────────────┬────────────────┬────────────────────────┬───────────────────┬────────┬──────────┬───────────────┬───────────────┬───────────┬────────────────┬───────────────┬───────────────────┬──────────────────────┬────────────────────────┬─────────────────────────┬──────────────────────┬────────────────────┬─────────────────────┬─────────────────────┬───────────────────────┬─────────────────────┬──────────────────────┐
│ site_id │ station_name │ latitude │ longitude │ huc_code │ drainage_area_sq_km │ is_reference_hcdn2009 │ elev_mean_m │ elev_max_m │ elev_min_m │ SLOPE_PCT │ ASPECT_NORTHNESS │ ASPECT_EASTNESS │ soil_water_capacity_avg │ soil_permeability_avg │ clay_pct_avg │ silt_pct_avg │ sand_pct_avg │ rock_dep

In [8]:
query = """
    SELECT *
    FROM flood_forecasting.main_marts.dim_sites AS a
    WHERE site_id = '05013500'
"""
conn.sql(query).show()

┌─────────┬──────────────┬──────────┬───────────┬─────────────────────┬──────────┐
│ site_id │ station_name │ latitude │ longitude │ drainage_area_sq_mi │ huc_code │
│ varchar │   varchar    │  double  │  double   │       double        │  int64   │
├─────────┴──────────────┴──────────┴───────────┴─────────────────────┴──────────┤
│                                     0 rows                                     │
└────────────────────────────────────────────────────────────────────────────────┘



**Conclusion: maybe delete dim_sites after merging drainage area info into dim_catchment_attributes**

In [9]:
query = """
SELECT site_id, latitude, longitude
FROM main_marts.dim_sites
"""
df = conn.sql(query).df()

fig = px.scatter_geo(df,
    lat='latitude',
    lon='longitude',
    scope='usa',
    title='USGS Sites'
)
fig.show()

# **Gage height**

In [10]:
query = """
SELECT site_id, latitude, longitude, observation_hour, gage_height_ft_mean
FROM flood_forecasting.main.flood_model
WHERE gage_height_ft_mean > 0
ORDER BY observation_hour
"""
df = conn.sql(query).df()

fig = px.scatter_geo(df,
    lat='latitude',
    lon='longitude',
    scope='usa',
    title='USGS Sites'
)
fig.show()

***While there are more locations in other table, this is the one that has the info***

In [11]:
df['time'] = df['observation_hour'].astype(str)

fig = px.scatter_geo(df,
    lat='latitude',
    lon='longitude',
    color='gage_height_ft_mean',
    animation_frame='time',
    scope='usa',
    title='Gage Height Over Time',
    color_continuous_scale='RdYlGn_r',
    range_color=[df['gage_height_ft_mean'].min(), df['gage_height_ft_mean'].max()],
    hover_data=['site_id']
)

fig.show()

***Maybe normalization?***

In [12]:
fig = px.line(df, x='observation_hour', y='gage_height_ft_mean', color='site_id', title='Gage Height Comparison')
fig.show()

In [13]:
fig = px.box(df, x='site_id', y='gage_height_ft_mean', title='Gage Height Distribution by Site')
fig.show()

In [14]:
query = """
    SELECT gage_height_ft_mean, streamflow_cfs_mean, precipitation_mm, 
           temperature_c, humidity_pct, drainage_area_sq_km
    FROM main.flood_model 
    WHERE gage_height_ft_mean > 0
"""

corr_df = conn.sql(query).df()

fig = px.imshow(corr_df.corr(), text_auto=True, title='Correlation Matrix', 
                color_continuous_scale='RdYlGn', zmin=-1, zmax=1)       
fig.show()

# **Streamflow**

In [15]:
query = """
SELECT site_id, latitude, longitude, observation_hour, streamflow_cfs_mean
FROM flood_forecasting.main.flood_model
WHERE streamflow_cfs_mean > 0
ORDER BY observation_hour
"""
df = conn.sql(query).df()

In [16]:
df['time'] = df['observation_hour'].astype(str)

fig = px.scatter_geo(df,
    lat='latitude',
    lon='longitude',
    color='streamflow_cfs_mean',
    animation_frame='time',
    scope='usa',
    title='Streamflow Over Time',
    color_continuous_scale='RdYlGn_r',
    range_color=[df['streamflow_cfs_mean'].min(), df['streamflow_cfs_mean'].max()],
    hover_data=['site_id']
)

fig.show()

In [17]:
fig = px.line(df, x='observation_hour', y='streamflow_cfs_mean', color='site_id', title='Streamflow Comparison')
fig.show()

In [ ]:
fig = px.box(df, x='site_id', y='streamflow_cfs_mean', title='Streamflow Distribution by Site')
fig.show()
# Test
